In [1]:
# Updated Vector DB Set Up and Upsert

import psycopg2
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone.grpc import PineconeGRPC as Pinecone

# Database connection details
DATABASE_HOST = "database-test-postgress-instance.cpk2uyae6iza.ap-south-1.rds.amazonaws.com"
DATABASE_USERNAME = "python_aiml"
DATABASE_PASSWORD = "python"
DATABASE_DB = "python_test_poc"
PORT = 5432

# Pinecone details
pinecone_api_key = "7844b232-6ba4-4aef-884b-7f826ec88d74"
index_name = "jagoai"
BATCH_SIZE = 200  # Adjust the batch size to avoid exceeding the size limit

# Function to connect to PostgreSQL database
def connect_to_db():
    try:
        conn = psycopg2.connect(
            dbname=DATABASE_DB,
            user=DATABASE_USERNAME,
            password=DATABASE_PASSWORD,
            host=DATABASE_HOST,
            port=PORT
        )
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to the database: {e}")
        raise

# Fetch schema with column names and data types, only including string types
def fetch_schema_with_data_types(conn):
    try:
        query = """
        SELECT table_name, column_name, data_type, character_maximum_length
        FROM information_schema.columns
        WHERE table_schema = 'public'
          AND (data_type = 'character varying' OR data_type IN ('text', 'varchar'))
          AND table_name IN ('issues', 'timesheet')
        ORDER BY table_name;

        """
        schema_df = pd.read_sql(query, conn)
        print(schema_df)
        return schema_df
    except Exception as e:
        print(f"Error fetching schema with data types: {e}")
        raise

# Fetch unique values from each column along with table details
def fetch_unique_values(conn, table_name, column_name):
    try:
        query = f"SELECT DISTINCT {column_name} FROM {table_name}"
        df = pd.read_sql(query, conn)
        return df[column_name].dropna().astype(str).tolist()
    except Exception as e:
        print(f"Error fetching unique values for {column_name} in {table_name}: {e}")
        return []

# Fetch all unique values for each column and map them to table details
def fetch_all_unique_values_with_table(conn, schema_df):
    unique_values_dict = {}
    for table_name in schema_df['table_name'].unique():
        unique_values_dict[table_name] = {}
        table_columns = schema_df[schema_df['table_name'] == table_name]
        for column_name in table_columns['column_name']:
            unique_values = fetch_unique_values(conn, table_name, column_name)
            unique_values_dict[table_name][column_name] = unique_values
    return unique_values_dict

# Initialize SentenceTransformer model for Hugging Face embeddings
def load_huggingface_model():
    model_name = 'sentence-transformers/all-mpnet-base-v2'
    embedding_model = SentenceTransformer(model_name)
    return embedding_model

# Generate embeddings for each unique value and store them
# Generate embeddings for each unique value and store them
def generate_and_store_embeddings(embedding_model, unique_values_dict):
    embeddings_dict = {}
    for table_name, columns in unique_values_dict.items():
        embeddings_dict[table_name] = {}
        for column_name, unique_values in columns.items():
            if unique_values:  # Check if there are any unique values
                try:
                    embeddings = embedding_model.encode(unique_values)
                    embeddings_dict[table_name][column_name] = {
                        "unique_values": unique_values,
                        "embeddings": embeddings
                    }
                except Exception as e:
                    print(f"Error generating embeddings for {column_name} in {table_name}: {e}")
                    embeddings_dict[table_name][column_name] = {
                        "unique_values": unique_values,
                        "embeddings": []  # Store an empty list if encoding fails
                    }
            else:
                print(f"No unique values found for {column_name} in {table_name}. Skipping embeddings.")
                embeddings_dict[table_name][column_name] = {
                    "unique_values": [],
                    "embeddings": []
                }
    return embeddings_dict


# Initialize Pinecone
def initialize_pinecone():
    pc = Pinecone(api_key=pinecone_api_key)
    index = pc.Index(index_name)
    return index

# Batch the embeddings for upserts
def batch_embeddings(upsert_data, batch_size):
    for i in range(0, len(upsert_data), batch_size):
        yield upsert_data[i:i + batch_size]

# Upsert embeddings into Pinecone with metadata for each table (namespace)
def upsert_embeddings_into_pinecone(index, embeddings_dict):
    for table_name, columns in embeddings_dict.items():
        for column_name, data in columns.items():
            upsert_data = []
            for i, embedding in enumerate(data['embeddings']):
                unique_value = data['unique_values'][i]
                vector_id = f"{table_name}_{column_name}_{i}"
                metadata = {"column_name": column_name, "unique_value": unique_value}

                upsert_data.append({
                    "id": vector_id,
                    "values": embedding.tolist(),
                    "metadata": metadata
                })

            # Batch the upsert to avoid exceeding size limits
            for batch in batch_embeddings(upsert_data, BATCH_SIZE):
                index.upsert(vectors=batch, namespace=table_name)
                print(f"Upserted batch for {column_name} in {table_name}")

# Main function to execute the process
def main():
    # Step 1: Connect to the database
    conn = connect_to_db()

    # Step 2: Fetch the schema with metadata and data types, only for string columns
    schema_df = fetch_schema_with_data_types(conn)
    print("Schema with string data types fetched successfully.")

    # Step 3: Fetch all unique values along with table details
    unique_values_dict = fetch_all_unique_values_with_table(conn, schema_df)
    print("Unique values for string columns fetched successfully.")

    # Step 4: Load the Hugging Face model for embeddings
    embedding_model = load_huggingface_model()
    print("Hugging Face model loaded successfully.")

    # Step 5: Generate embeddings for all unique values
    embeddings_dict = generate_and_store_embeddings(embedding_model, unique_values_dict)
    print("Embeddings for string columns generated successfully.")

    # Step 6: Initialize Pinecone and upsert embeddings under each table's namespace
    pinecone_index = initialize_pinecone()
    upsert_embeddings_into_pinecone(pinecone_index, embeddings_dict)
    print("Embeddings upserted into Pinecone successfully.")

if __name__ == "__main__":
    main()

In [8]:
#Set Up Vector DB Based On Increamental Data.

import psycopg2
import pandas as pd
from datetime import datetime
from sentence_transformers import SentenceTransformer
from pinecone.grpc import PineconeGRPC as Pinecone


class DatabaseManager:
    def __init__(self, host, username, password, dbname, port):
        self.host = host
        self.username = username
        self.password = password
        self.dbname = dbname
        self.port = port
        self.connection = self.connect_to_db()

    def connect_to_db(self):
        try:
            conn = psycopg2.connect(
                dbname=self.dbname,
                user=self.username,
                password=self.password,
                host=self.host,
                port=self.port
            )
            print("Successfully connected to the database.")
            return conn
        except psycopg2.Error as e:
            print(f"Error connecting to the database: {e}")
            raise

    def fetch_schema_with_data_types(self):
        try:
            query = """
            SELECT table_name, column_name, data_type, character_maximum_length
            FROM information_schema.columns
            WHERE table_schema = 'public'
            ORDER BY table_name;
            """
            schema_df = pd.read_sql(query, self.connection)
            print("Fetched schema with data types successfully.")
            return schema_df
        except Exception as e:
            print(f"Error fetching schema: {e}")
            raise

    def fetch_unique_values_by_modified_date(self, table_name, column_name, modified_columns):
        # Get the current date and time in MM-DD-YYYY HH:MM format
        current_datetime = datetime.now().strftime('%m-%d-%Y %H:%M')
        print(f"Fetching unique values for {column_name} in {table_name} for date: {current_datetime}")
        
        # Check which modified columns exist in the current table schema
        existing_modified_columns = [
            col for col in modified_columns if col in self.fetch_schema_with_data_types().query(f'table_name == "{table_name}"')['column_name'].values
        ]
        
        if not existing_modified_columns:
            print(f"No valid modified columns found for table: {table_name}")
            return []  # Return empty if no valid column is found
    
        for modified_column in existing_modified_columns:
            try:
                query = f"""
                SELECT DISTINCT {column_name}
                FROM {table_name}
                WHERE TO_CHAR("{modified_column}", 'MM-DD-YYYY HH24:MI') = %s;
                """
                df = pd.read_sql(query, self.connection, params=(current_datetime,))
                print(f"Fetched unique values for {column_name}: {df[column_name].tolist()}")
                return df[column_name].dropna().astype(str).tolist()
            except Exception as e:
                print(f"Error fetching unique values for {column_name} in {table_name} using {modified_column}: {e}")
                continue
        return []  # Return empty if no valid column is found
    

class EmbeddingGenerator:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.model = self.load_huggingface_model(model_name)

    def load_huggingface_model(self, model_name):
        print(f"Loading model: {model_name}")
        return SentenceTransformer(model_name)

    def generate_embeddings(self, unique_values):
        try:
            embeddings = self.model.encode(unique_values)
            print(f"Generated embeddings for {len(unique_values)} unique values.")
            return embeddings
        except Exception as e:
            print(f"Error generating embeddings: {e}")
            return []


class PineconeManager:
    def __init__(self, api_key, index_name, batch_size=200):
        self.api_key = api_key
        self.index_name = index_name
        self.batch_size = batch_size
        self.index = self.initialize_pinecone()

    def initialize_pinecone(self):
        pc = Pinecone(api_key=self.api_key)
        index = pc.Index(self.index_name)
        print(f"Pinecone index '{self.index_name}' initialized.")
        return index

    def check_if_exists(self, table_name, column_name, unique_value):
        vector_id = f"{table_name}_{column_name}_{unique_value}"
        try:
            result = self.index.fetch([vector_id], namespace=table_name)
            exists = bool(result["vectors"])
            print(f"Vector existence check for {vector_id}: {exists}")
            return exists
        except Exception as e:
            print(f"Error checking existence in Pinecone: {e}")
            return False

    def batch_embeddings(self, upsert_data):
        for i in range(0, len(upsert_data), self.batch_size):
            yield upsert_data[i:i + self.batch_size]

    def upsert_embeddings(self, embeddings_dict):
        for table_name, columns in embeddings_dict.items():
            for column_name, data in columns.items():
                upsert_data = []
                for i, embedding in enumerate(data['embeddings']):
                    unique_value = data['unique_values'][i]

                    # Check if the vector already exists
                    if self.check_if_exists(table_name, column_name, unique_value):
                        print(f"Vector for {unique_value} already exists. Skipping.")
                        continue

                    vector_id = f"{table_name}_{column_name}_{i}"
                    metadata = {"column_name": column_name, "unique_value": unique_value}

                    upsert_data.append({
                        "id": vector_id,
                        "values": embedding.tolist(),
                        "metadata": metadata
                    })

                # Batch the upsert to avoid exceeding size limits
                for batch in self.batch_embeddings(upsert_data):
                    self.index.upsert(vectors=batch, namespace=table_name)
                    print(f"Upserted batch for {column_name} in {table_name}")


class DataProcessor:
    def __init__(self, db_manager, embedding_generator, pinecone_manager):
        self.db_manager = db_manager
        self.embedding_generator = embedding_generator
        self.pinecone_manager = pinecone_manager

        # Possible last modified column names
        self.modified_columns = ['last_modified_time', 'last_updated_time', 'last_modified_date']

    def process_data(self):
        schema_df = self.db_manager.fetch_schema_with_data_types()

        # Verify which modified columns exist in the schema
        existing_modified_columns = []
        for modified_column in self.modified_columns:
            if modified_column in schema_df['column_name'].values:
                existing_modified_columns.append(modified_column)

        print(f"Using modified columns: {existing_modified_columns}")

        embeddings_dict = {}
        for table_name in schema_df['table_name'].unique():
            embeddings_dict[table_name] = {}
            table_columns = schema_df[schema_df['table_name'] == table_name]

            for column_name in table_columns['column_name']:
                unique_values = self.db_manager.fetch_unique_values_by_modified_date(
                    table_name, column_name, existing_modified_columns)

                if unique_values:
                    embeddings = self.embedding_generator.generate_embeddings(unique_values)
                    embeddings_dict[table_name][column_name] = {
                        "unique_values": unique_values,
                        "embeddings": embeddings
                    }

        self.pinecone_manager.upsert_embeddings(embeddings_dict)


if __name__ == "__main__":
    # Initialize Database, Embedding Generator, and Pinecone Manager
    db_manager = DatabaseManager(
        host="database-test-postgress-instance.cpk2uyae6iza.ap-south-1.rds.amazonaws.com",
        username="python_aiml",
        password="python",
        dbname="python_test_poc",
        port=5432
    )
    
    embedding_generator = EmbeddingGenerator()
    
    pinecone_manager = PineconeManager(
        api_key="7844b232-6ba4-4aef-884b-7f826ec88d74",
        index_name="jagoai"
    )

    # Create DataProcessor to orchestrate the process
    data_processor = DataProcessor(db_manager, embedding_generator, pinecone_manager)
    data_processor.process_data()


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Successfully connected to the database.
Loading model: sentence-transformers/all-mpnet-base-v2


INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['C:\\Users\\Basavaraj\\AppData\\Local\\anaconda3\\Lib\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into PineconeGRPC
C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users

Pinecone index 'jagoai' initialized.
Fetched schema with data types successfully.
Using modified columns: ['last_modified_time', 'last_updated_time', 'last_modified_date']
Fetching unique values for status_id in issues for date: 10-21-2024 12:00
Fetched schema with data types successfully.
Fetched schema with data types successfully.
Fetched schema with data types successfully.
Fetched unique values for status_id: []
Fetching unique values for flag in issues for date: 10-21-2024 12:00
Fetched schema with data types successfully.
Fetched schema with data types successfully.
Fetched schema with data types successfully.
Fetched unique values for flag: []
Fetching unique values for time_left in issues for date: 10-21-2024 12:00
Fetched schema with data types successfully.
Fetched schema with data types successfully.
Fetched schema with data types successfully.


C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.connection, params=(current_datetime,))
C:\Users\B

...

C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.connection, params=(current_datetime,))
C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users\B

Fetched schema with data types successfully.
Fetched unique values for zuid: []
Fetching unique values for created_time in users for date: 10-21-2024 12:00
Fetched schema with data types successfully.
Fetched schema with data types successfully.
Fetched schema with data types successfully.


C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.connection, params=(current_datetime,))
C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users\B

Fetched unique values for created_time: []
Fetching unique values for user_email in users for date: 10-21-2024 12:00
Fetched schema with data types successfully.
Fetched schema with data types successfully.
Fetched schema with data types successfully.
Fetched unique values for user_email: []


C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users\Basavaraj\AppData\Local\Temp\ipykernel_15564\3907896313.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql(query, self.connection)
C:\Users\Basavaraj\AppData\Loca

In [ ]:
#Scheduler For Incremental Data.

from apscheduler.schedulers.blocking import BlockingScheduler
from datetime import datetime

# Your previously defined classes (DatabaseManager, EmbeddingGenerator, PineconeManager, DataProcessor) go here

def run_process_data_job():
    print(f"Process data job started at: {datetime.now()}")
    
    # Initialize Database, Embedding Generator, and Pinecone Manager
    db_manager = DatabaseManager(
        host="database-test-postgress-instance.cpk2uyae6iza.ap-south-1.rds.amazonaws.com",
        username="python_aiml",
        password="python",
        dbname="python_test_poc",
        port=5432
    )
    
    embedding_generator = EmbeddingGenerator()
    
    pinecone_manager = PineconeManager(
        api_key="7844b232-6ba4-4aef-884b-7f826ec88d74",
        index_name="jagoai"
    )

    # Create DataProcessor to orchestrate the process
    data_processor = DataProcessor(db_manager, embedding_generator, pinecone_manager)
    data_processor.process_data()
    
    print(f"Process data job finished at: {datetime.now()}")

# Setup APScheduler
scheduler = BlockingScheduler()

# Schedule the job to run every 3 hours
scheduler.add_job(run_process_data_job, 'interval', hours=3)

# Start the scheduler
scheduler.start()


INFO:apscheduler.scheduler:Adding job tentatively -- it will be properly scheduled when the scheduler starts
INFO:apscheduler.scheduler:Added job "run_process_data_job" to job store "default"
INFO:apscheduler.scheduler:Scheduler started


In [12]:
# pip install apscheduler

   ---------------------------------------- 0.0/59.3 kB ? eta -:--:--
   ---------------------------------- ----- 51.2/59.3 kB ? eta -:--:--
   ---------------------------------------- 59.3/59.3 kB 791.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
